# 目的

* torchのtrainループ書くのだるいな〜ってときに。

* datasetの定義 → dataloaderの定義 → trainループを書く　

                   ⇓    skorchを使うと
                   
""""""""""""""""""""""""""""""""""""""""""""""""

   　　　　clf.fit(x_train, y_train)
                
""""""""""""""""""""""""""""""""""""""""""""""""

* 参考　（というかほぼこれ）

https://www.pytry3g.com/entry/skorch-tutorial

* 実行確認環境

    torch==1.3.1
    
    scikit-learn==0.21.3
    
    skorch==0.7.0

# 特徴量準備およびネットワークの定義

In [13]:
import numpy as np
import pickle
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset

In [3]:
wine = load_wine()
X = wine.data.astype(np.float32)
y = wine.target.astype(np.int64)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=wine.target, random_state=0)

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(13, 30)
        self.fc2 = nn.Linear(30, 30)
        self.fc3 = nn.Linear(30, 3)
    
    def forward(self, x):
        x = F.relu(self.fc1(x)) 
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# skorchを使ってみよう

In [5]:
from skorch import NeuralNetClassifier

In [6]:
clf = NeuralNetClassifier(
    Net,
    optimizer = torch.optim.SGD,
    criterion = nn.CrossEntropyLoss,
    max_epochs = 100,
    lr = 0.001,
    iterator_train__batch_size=32, # default=128
    iterator_train__shuffle=True
)

In [7]:
clf.fit(X_train, y_train)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1       26.7046       0.3333        4.3661  0.0763
      2        4.9759       0.3333        3.7588  0.0038
      3        4.6667       0.4000        1.4078  0.0037
      4        1.4011       0.7333        0.7109  0.0027
      5        1.7998       0.2667        1.8648  0.0027
      6        1.2832       0.5333        0.8326  0.0028
      7        0.8703       0.5667        0.7577  0.0027
      8        0.9239       0.6000        0.8386  0.0026
      9        0.9376       0.6333        0.8805  0.0026
     10        0.9093       0.6667        0.7351  0.0026
     11        0.7793       0.7333        0.6264  0.0027
     12        0.7676       0.7333        0.6042  0.0026
     13        0.7524       0.7333        0.6466  0.0031
     14        0.7710       0.5667        0.8324  0.0030
     15        1.0122       0.7333        0.6705  0.0032
     16        0.8152       0.7

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=Net(
    (fc1): Linear(in_features=13, out_features=30, bias=True)
    (fc2): Linear(in_features=30, out_features=30, bias=True)
    (fc3): Linear(in_features=30, out_features=3, bias=True)
  ),
)

In [8]:
y_pred = clf.predict(X_test)

In [11]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.90      0.75      0.82        12
           1       0.67      0.86      0.75        14
           2       0.50      0.40      0.44        10

    accuracy                           0.69        36
   macro avg       0.69      0.67      0.67        36
weighted avg       0.70      0.69      0.69        36



# モデルの保存

In [15]:
with open('samplenn.pkl', mode='wb') as f:
    pickle.dump(clf, f)

/home/yosuke-kurosu/.local/lib/python3.6/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/yosuke-kurosu/.local/lib/python3.6/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Linear. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/yosuke-kurosu/.local/lib/python3.6/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type CrossEntropyLoss. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [17]:
model = pickle.load(open("samplenn.pkl", 'rb'))

# GPUの使用

In [20]:
clf = NeuralNetClassifier(
    Net,
    optimizer = torch.optim.SGD,
    criterion = nn.CrossEntropyLoss,
    max_epochs = 100,
    lr = 0.001,
    iterator_train__batch_size=32, # default=128
    iterator_train__shuffle=True,
    device = "cuda" # オプションにCUDAを追加する
)

In [21]:
clf.fit(X_train, y_train)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        9.7851       0.4000        3.8554  0.1083
      2        4.6137       0.5000        2.3664  0.0137
      3        2.3092       0.4000        2.5766  0.0041
      4        2.3896       0.4000        1.5512  0.0042
      5        1.9207       0.4000        3.4217  0.0042
      6        2.0388       0.6667        0.7240  0.0042
      7        0.8315       0.6333        0.8974  0.0047
      8        0.9262       0.4000        1.8385  0.0051
      9        1.5752       0.4000        1.4823  0.0040
     10        1.0791       0.7333        0.6428  0.0037
     11        0.7826       0.7000        0.6475  0.0047
     12        0.8909       0.7333        0.7581  0.0040
     13        1.1138       0.5333        0.8386  0.0039
     14        0.9933       0.6000        0.6889  0.0045
     15        0.9210       0.7333        0.6372  0.0043
     16        0.7633       0.4

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=Net(
    (fc1): Linear(in_features=13, out_features=30, bias=True)
    (fc2): Linear(in_features=30, out_features=30, bias=True)
    (fc3): Linear(in_features=30, out_features=3, bias=True)
  ),
)

# パイプラインの使用

In [23]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
pipe = Pipeline([
    ('scale', StandardScaler()),
    ('clf', clf)
])

pipe.fit(X_train, y_train)

Re-initializing module.
Re-initializing optimizer.
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        1.1489       0.2333        1.1608  0.0042
      2        1.1483       0.2333        1.1602  0.0046
      3        1.1476       0.2333        1.1595  0.0047
      4        1.1469       0.2333        1.1588  0.0044
      5        1.1463       0.2333        1.1582  0.0044
      6        1.1457       0.2333        1.1575  0.0046
      7        1.1450       0.2333        1.1569  0.0048
      8        1.1444       0.2333        1.1562  0.0046
      9        1.1437       0.2333        1.1556  0.0049
     10        1.1431       0.2333        1.1549  0.0049
     11        1.1424       0.2333        1.1542  0.0053
     12        1.1417       0.2333        1.1536  0.0040
     13        1.1411       0.2333        1.1530  0.0044
     14        1.1405       0.2333        1.1524  0.0045
     15        1.1399       0.2333   

Pipeline(memory=None,
         steps=[('scale',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('clf',
                 <class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=Net(
    (fc1): Linear(in_features=13, out_features=30, bias=True)
    (fc2): Linear(in_features=30, out_features=30, bias=True)
    (fc3): Linear(in_features=30, out_features=3, bias=True)
  ),
))],
         verbose=False)